# Seq2Seq Example. - building Kor-En Translator

## Strategy
- Pretrain Decoder - Endcoder for each dataset (Korean, English)
  - Korean Encoder Training
  - English Decoder Training
  - English Encoder Training
  - Korean Decoder Training
- End-to-End Training - parallel corpus text dataset
  - Korean - English Translator
  - English - Korean Translator
- 


In [1]:
from model import GRUEncoder, get_default_tokenizer
from data import KoEnParallel, Korean, English, Wikipedia
from transformers import AutoTokenizer, GPT2TokenizerFast
from torch.utils import data

# ko_tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
tokenizer = get_default_tokenizer("en")
data = Wikipedia()
print(tokenizer.decode(tokenizer.pad_token_id))

ko_dataset = English(tokenizer, 1024, 'train')
dataloader = data.DataLoader(ko_dataset, batch_size=10, collate_fn=ko_dataset)
ko_encoder = GRUEncoder(len(tokenizer.vocab), 1024, 1024, 2, padding_id=tokenizer.pad_token_id)

next(iter(dataloader))

token added 1


Using custom data configuration 20230201.ko-date=20230201,language=ko


ValueError: Builder wikipedia is not streamable.

In [4]:
import pytorch_lightning as pl
from pytorch_lightning import loggers, callbacks

logger = loggers.TensorBoardLogger('logs')
model_checkout = callbacks.ModelCheckpoint('model/encoder', filename='model-{epoch}-{val_loss:.3f}')

trainer = pl.Trainer(logger=logger, accelerator='gpu', callbacks=[model_checkout])
trainer.fit(model=ko_encoder, train_dataloaders=dataloader, val_dataloaders=dataloader)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:411: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 52.4 M
1 | enc       | GRU              | 31.5 M
2 | fc        | LazyLinear       | 0     
3 | loss

Sanity Checking: 0it [00:00, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


IndexError: too many indices for tensor of dimension 2

In [7]:
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
from data import get_default_tokenizer
tokenizer = get_default_tokenizer('en')
enc = tokenizer.encode("하늘에 계신 우리 아버지 아버지의 이름이 거룩히 빛나시며 ")
tokenizer.convert_tokens_to_string(enc)

Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import Dataset

from_data = Dataset.from_dict({"v": ["afeo.jfaew.fjaw.lifja.ilef.jli", "afeo.jfaew.fjaw.lifja.ilef.jli"]})

to_data= from_data.map(lambda x: {"vv": x.split('.')}, input_columns='v', remove_columns=['v'])


# data = Wikipedia()

  0%|          | 0/2 [00:00<?, ?ex/s]

In [2]:
from model import EncoderDecoder
from torch.utils import data
from torchtext import functional as TF
from model import EncoderDecoder
from data import get_default_tokenizer, KoEnParallel, ParallelCollator

ko_tokenizer = get_default_tokenizer('ko')
en_tokenizer = get_default_tokenizer('en')


collator = ParallelCollator(from_tokenizer=ko_tokenizer, 
                            to_tokenizer=en_tokenizer, 
                            from_pad_id=ko_tokenizer.encode('<pad>')[0], 
                            to_pad_id=en_tokenizer.encode('<pad>')[0], 
                            from_eos_id=ko_tokenizer.encode('<eos>')[0], 
                            to_bos_id=en_tokenizer.encode('<bos>')[0], 
                            to_eos_id=en_tokenizer.encode('<eos>')[0])

para_dataset = KoEnParallel(split='train')
train_dataloader = data.DataLoader(dataset=para_dataset, collate_fn=collator, batch_size=10)
seq2seq_trans = EncoderDecoder()


ko_batch, en_batch = next(iter(train_dataloader))
seq2seq_trans(ko_batch, en_batch)


Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


TypeError: EncoderDecoder.forward() missing 1 required positional argument: 'dec_X'